In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import re

df = pd.read_csv('Data Final_Employee Attrition.csv')
df = df.rename(columns={'BusinessTravel':'Business Travel'})
df = df.rename(columns=lambda col: re.sub(r"\s+", "_", col.lower()))
df = df.drop(columns=['employee_number', 'over_18', 'standard_hours'])
df = df.dropna()
df

,age,attrition,business_travel,daily_rate,department,distance_from_home,education,education_field,environment_satisfaction,gender,...,performance_rating,relationship_satisfaction,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_in_current_role,years_since_last_promotion,years_with_current_manager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,Female,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,Male,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,4,Male,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,Female,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,Male,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,3,Male,...,3,3,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,4,Male,...,3,1,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,2,Male,...,4,2,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,4,Male,...,3,4,0,17,3,2,9,6,0,8


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Display summary statistics
display(df.describe())

,age,daily_rate,distance_from_home,education,environment_satisfaction,hourly_rate,job_involvement,job_level,job_satisfaction,monthly_income,...,performance_rating,relationship_satisfaction,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_in_current_role,years_since_last_promotion,years_with_current_manager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,2.721769,65.891156,2.729932,2.063946,2.728571,6502.931293,...,3.153741,2.712245,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,1.093082,20.329428,0.711561,1.106940,1.102846,4707.956783,...,0.360824,1.081209,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,1009.000000,...,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,2.000000,48.000000,2.000000,1.000000,2.000000,2911.000000,...,3.000000,2.000000,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,3.000000,66.000000,3.000000,2.000000,3.000000,4919.000000,...,3.000000,3.000000,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,4.000000,83.750000,3.000000,3.000000,4.000000,8379.000000,...,3.000000,4.000000,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,4.000000,100.000000,4.000000,5.000000,4.000000,19999.000000,...,4.000000,4.000000,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [39]:
#Mapping binary variables
df['over_time'] = df['over_time'].map({'Yes': 1, 'No': 0})
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})
df['attrition'] = df['attrition'].map({'Yes': 1, 'No': 0})
# Convert categorical variables to numerical using one-hot encoding
df = pd.get_dummies(df, columns=['business_travel', 'department', 'education_field', 'job_role', 'marital_status'], drop_first=True)
df = df.rename(columns=lambda col: re.sub(r"\s+", "_", col.lower()))

df

,age,attrition,daily_rate,distance_from_home,education,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,...,job_role_human_resources,job_role_laboratory_technician,job_role_manager,job_role_manufacturing_director,job_role_research_director,job_role_research_scientist,job_role_sales_executive,job_role_sales_representative,marital_status_married,marital_status_single
0,41,1,1102,1,2,2,0,94,3,2,...,0,0,0,0,0,0,1,0,0,1
1,49,0,279,8,1,3,1,61,2,2,...,0,0,0,0,0,1,0,0,1,0
2,37,1,1373,2,2,4,1,92,2,1,...,0,1,0,0,0,0,0,0,0,1
3,33,0,1392,3,4,4,0,56,3,1,...,0,0,0,0,0,1,0,0,1,0
4,27,0,591,2,1,1,1,40,3,1,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,0,884,23,2,3,1,41,4,2,...,0,1,0,0,0,0,0,0,1,0
1466,39,0,613,6,1,4,1,42,2,3,...,0,0,0,0,0,0,0,0,1,0
1467,27,0,155,4,3,2,1,87,4,2,...,0,0,0,1,0,0,0,0,1,0
1468,49,0,1023,2,3,4,1,63,2,2,...,0,0,0,0,0,0,1,0,1,0


In [40]:
#Setting 'attrition' as the target variable
X = df.drop('attrition', axis=1)
y = df['attrition']

#Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
X_train.describe()

,age,daily_rate,distance_from_home,education,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,job_satisfaction,...,job_role_human_resources,job_role_laboratory_technician,job_role_manager,job_role_manufacturing_director,job_role_research_director,job_role_research_scientist,job_role_sales_executive,job_role_sales_representative,marital_status_married,marital_status_single
count,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,...,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000
mean,36.774660,799.410714,9.260204,2.896259,2.695578,0.601190,66.671769,2.721939,2.028061,2.735544,...,0.035714,0.173469,0.067177,0.090986,0.057823,0.205782,0.215986,0.063776,0.467687,0.305272
std,9.203851,405.727156,8.153860,1.038552,1.088828,0.489862,20.463674,0.713442,1.103215,1.097290,...,0.185656,0.378814,0.250435,0.287712,0.233508,0.404444,0.411680,0.244457,0.499167,0.460719
min,18.000000,102.000000,1.000000,1.000000,1.000000,0.000000,30.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,464.000000,2.000000,2.000000,2.000000,0.000000,48.000000,2.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,36.000000,796.500000,7.000000,3.000000,3.000000,1.000000,67.000000,3.000000,2.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,43.000000,1157.250000,14.000000,4.000000,4.000000,1.000000,84.000000,3.000000,3.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,60.000000,1496.000000,29.000000,5.000000,4.000000,1.000000,100.000000,4.000000,5.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [42]:
X_test.describe()

,age,daily_rate,distance_from_home,education,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,job_satisfaction,...,job_role_human_resources,job_role_laboratory_technician,job_role_manager,job_role_manufacturing_director,job_role_research_director,job_role_research_scientist,job_role_sales_executive,job_role_sales_representative,marital_status_married,marital_status_single
count,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,...,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000
mean,37.520408,814.785714,8.921769,2.979592,2.826531,0.595238,62.768707,2.761905,2.207483,2.700680,...,0.034014,0.187075,0.078231,0.129252,0.040816,0.170068,0.244898,0.027211,0.418367,0.377551
std,8.846313,394.952442,7.923956,0.963281,1.105606,0.491683,19.506564,0.704286,1.112055,1.126261,...,0.181573,0.390636,0.268993,0.336050,0.198202,0.376333,0.430760,0.162975,0.494132,0.485601
min,18.000000,106.000000,1.000000,1.000000,1.000000,0.000000,30.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,31.000000,471.750000,2.000000,2.000000,2.000000,0.000000,46.000000,2.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,36.000000,828.000000,7.000000,3.000000,3.000000,1.000000,62.000000,3.000000,2.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,44.000000,1155.500000,13.750000,4.000000,4.000000,1.000000,78.000000,3.000000,3.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,60.000000,1499.000000,29.000000,5.000000,4.000000,1.000000,100.000000,4.000000,5.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [43]:
# Feature Scaling using Min-Max Scaler
from sklearn.preprocessing import MinMaxScaler

numerical_cols = ['age', 'daily_rate', 'distance_from_home', 'hourly_rate', 'monthly_income', 'monthly_rate',
                    'num_companies_worked', 'percent_salary_hike', 'total_working_years', 'training_times_last_year',
                    'years_at_company', 'years_in_current_role', 'years_since_last_promotion', 'years_with_current_manager']

scaler = MinMaxScaler()

columns = X_train.columns

X_train_scaled = X_train.copy()
X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])

X_test_scaled = X_test.copy()
X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [44]:
display(X_train_scaled)

,age,daily_rate,distance_from_home,education,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,job_satisfaction,...,job_role_human_resources,job_role_laboratory_technician,job_role_manager,job_role_manufacturing_director,job_role_research_director,job_role_research_scientist,job_role_sales_executive,job_role_sales_representative,marital_status_married,marital_status_single
1097,0.142857,0.177905,0.714286,2,3,1,0.385714,2,1,1,...,0,1,0,0,0,0,0,0,0,0
727,0.000000,0.132712,0.142857,2,2,1,0.614286,3,1,4,...,0,0,0,0,0,1,0,0,0,1
254,0.261905,0.821377,0.678571,2,4,1,0.214286,3,2,4,...,0,0,0,0,0,0,1,0,0,0
1175,0.500000,0.279770,0.392857,3,4,1,0.514286,3,2,2,...,0,0,0,1,0,0,0,0,1,0
1341,0.309524,0.149928,0.678571,3,2,1,0.842857,3,2,3,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,0.404762,0.464849,0.964286,3,2,1,0.228571,4,2,3,...,0,1,0,0,0,0,0,0,1,0
1294,0.547619,0.247489,0.142857,3,2,1,0.785714,4,2,2,...,0,0,0,0,0,0,0,0,0,1
860,0.095238,0.827834,0.071429,4,3,1,0.257143,2,1,4,...,0,0,0,0,0,1,0,0,1,0
1459,0.261905,0.915352,0.428571,2,4,1,0.228571,2,2,2,...,0,1,0,0,0,0,0,0,1,0


In [45]:
display(X_test_scaled)

,age,daily_rate,distance_from_home,education,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,job_satisfaction,...,job_role_human_resources,job_role_laboratory_technician,job_role_manager,job_role_manufacturing_director,job_role_research_director,job_role_research_scientist,job_role_sales_executive,job_role_sales_representative,marital_status_married,marital_status_single
1041,0.238095,0.548063,0.142857,3,4,1,0.771429,3,2,1,...,0,0,0,0,0,0,1,0,0,1
184,0.833333,0.704448,0.428571,2,4,0,0.385714,4,2,1,...,0,0,0,1,0,0,0,0,0,0
1222,0.142857,0.098996,0.750000,1,4,1,0.400000,1,1,3,...,1,0,0,0,0,0,0,0,1,0
67,0.642857,0.887374,0.214286,3,2,1,0.414286,3,3,1,...,0,0,0,0,0,1,0,0,0,0
220,0.428571,0.928264,0.142857,2,4,1,0.457143,3,2,2,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,0.380952,0.144907,0.035714,3,4,1,0.428571,3,2,4,...,0,0,0,0,0,0,1,0,0,1
560,0.380952,0.046628,0.250000,5,2,0,0.028571,3,2,1,...,0,0,0,1,0,0,0,0,0,0
945,0.761905,0.875179,0.964286,3,4,0,0.185714,3,4,1,...,0,0,0,0,1,0,0,0,1,0
522,0.452381,0.805595,0.321429,2,4,1,0.714286,4,1,4,...,0,0,0,0,0,1,0,0,0,1


In [46]:
# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, output_dict=True)

# Accuracy and confusion matrix
accuracy_df = pd.DataFrame({'Accuracy': [accuracy]})
display(accuracy_df)

conf_matrix_df = pd.DataFrame(conf_matrix, columns=['Predicted No', 'Predicted Yes'], index=['Actual No', 'Actual Yes'])
display(conf_matrix_df)

# Classification report
classification_df = pd.DataFrame(classification_rep).transpose()
display(classification_df)

#Logistic Regression Model Result
X_train_scaled_reset = X_train_scaled.reset_index(drop=True)
y_train_reset = y_train.reset_index(drop=True)

X_train_with_const = sm.add_constant(X_train_scaled_reset)

logit_model = sm.Logit(y_train_reset, X_train_with_const)
result = logit_model.fit()

print(result.summary())

,Accuracy
0,0.894558


,Predicted No,Predicted Yes
Actual No,248,7
Actual Yes,24,15


,precision,recall,f1-score,support
0,0.911765,0.972549,0.941176,255.000000
1,0.681818,0.384615,0.491803,39.000000
accuracy,0.894558,0.894558,0.894558,0.894558
macro avg,0.796791,0.678582,0.716490,294.000000
weighted avg,0.881262,0.894558,0.881566,294.000000


         Current function value: 0.286452
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:              attrition   No. Observations:                 1176
Model:                          Logit   Df Residuals:                     1131
Method:                           MLE   Df Model:                           44
Date:                Wed, 03 Jan 2024   Pseudo R-squ.:                  0.3681
Time:                        13:40:26   Log-Likelihood:                -336.87
converged:                      False   LL-Null:                       -533.07
Covariance Type:            nonrobust   LLR p-value:                 1.901e-57
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                -6.1171     25.987     -0.235      0.814     -57.051      44.

c:\Users\Admin\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [47]:
# Checking Multicollinearity with VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["Variable"] = X_train_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_with_const.values, i) for i in range(X_train_with_const.shape[1])]

print(vif_data)

                             Variable         VIF
0                               const  512.827622
1                                 age    2.150523
2                          daily_rate    1.037287
3                  distance_from_home    1.036653
4                           education    1.108628
5            environment_satisfaction    1.041202
6                              gender    1.037298
7                         hourly_rate    1.029129
8                     job_involvement    1.039074
9                           job_level   14.776370
10                   job_satisfaction    1.033665
11                     monthly_income   18.516165
12                       monthly_rate    1.032907
13               num_companies_worked    1.295086
14                          over_time    1.045977
15                percent_salary_hike    2.536125
16                 performance_rating    2.521769
17          relationship_satisfaction    1.031562
18                 stock_option_level    1.859904
